In [39]:
import requests # Python package to send get/post requests to a webpage
import os       # Python package to interface with the operating system
import pandas as pd
import csv
import json
from operator import itemgetter
from collections import Counter

In [20]:
def get_current_ids(file_path):

    with open(file_path, 'r') as f:
        current_recipe_ids = csv.reader(f)
        current_recipe_ids = list(current_recipe_ids)[0]
        current_recipe_ids = set([int(id) for id in current_recipe_ids])
    return current_recipe_ids

In [3]:
def get_random_recipes(number, payload):
    '''
    This endpoint pulls the full recipes.
    '''
    endpoint = "https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/recipes/random?number=" + str(number)
    response = requests.get(endpoint, headers=payload)
    recipe_text = response.text.replace('},{"vegetarian' , '}$%&->recipe_end<-&%${"vegetarian')
    recipe_json = response.json()['recipes']
    recipe_ids = [recipe['id'] for recipe in recipe_json]

    return recipe_text[12:-2], recipe_json, recipe_ids, response

In [52]:
def remove_duplicate_recipes(current_recipe_ids, recipe_json, recipe_ids):
    for idx, id in enumerate(recipe_ids):
        if id in current_recipe_ids:
            del recipe_json[idx]
            del recipe_ids[idx]
            continue

    recipe_text = json.dumps(recipe_json).replace('}, {"vegetarian' , '}$%&->recipe_end<-&%${"vegetarian')[1:-1]
    

    return recipe_text, recipe_ids

In [5]:
API_KEY = os.environ['Spoontacular_API_KEY']
payload = {'X-RapidAPI-Key': API_KEY}
folder_path = '/home/katie/01-OneDrive/01_galvanize_dsi/capstones/03-capstone_3/data/'
text_files = ['random_recipes.txt', 'recipe_ids.csv']

#can only pull up to 100 in one call to API
recipe_text, recipe_json, recipe_ids, response = get_random_recipes(number = 2, payload = payload)

In [21]:
current_recipe_ids = get_current_ids('/home/katie/01-OneDrive/01_galvanize_dsi/capstones/03-capstone_3/data/recipe_ids.csv')

In [53]:
recipe_text, recipe_ids = remove_duplicate_recipes(current_recipe_ids, recipe_json, recipe_ids)

In [54]:
recipe_text

'{"vegetarian": true, "vegan": false, "glutenFree": false, "dairyFree": false, "veryHealthy": false, "cheap": false, "veryPopular": true, "sustainable": false, "weightWatcherSmartPoints": 4, "gaps": "no", "lowFodmap": false, "ketogenic": false, "whole30": false, "preparationMinutes": 0, "cookingMinutes": 45, "sourceUrl": "http://www.MotherThyme.com/2011/09/whole-wheat-pumpkin-pancakes.html", "spoonacularSourceUrl": "https://spoonacular.com/whole-wheat-pumpkin-pancakes-515428", "aggregateLikes": 1936, "spoonacularScore": 62.0, "healthScore": 7.0, "pricePerServing": 34.48, "extendedIngredients": [{"id": 18371, "aisle": "Baking", "image": "white-powder.jpg", "consitency": "solid", "name": "baking powder", "original": "3 teaspoons baking powder", "originalString": "3 teaspoons baking powder", "originalName": "baking powder", "amount": 3.0, "unit": "teaspoons", "meta": [], "metaInformation": [], "measures": {"us": {"amount": 3.0, "unitShort": "tsps", "unitLong": "teaspoons"}, "metric": {"am

In [55]:
def open_files(file_path, recipe_file, ids_file):
    recipe_file = open(file_path + recipe_file, 'r')
    ids_file = open(file_path + ids_file, 'r')
    return recipe_file.read(), ids_file.read()


def convert_to_json(recipes_text, ids_text):

    recipe_ids = list(map(lambda x: int(x), ids_text.split(", ")))
    recipes_list = recipes_text.split("$%&->recipe_end<-&%$")
    recipes_json = []
    for recipe in recipes_list:
        recipes_json.append(json.loads(recipe))
    return recipes_json, recipe_ids

In [56]:
file_path = '/home/katie/01-OneDrive/01_galvanize_dsi/capstones/03-capstone_3/data/'
recipes_text , ids_text = open_files(file_path, recipe_file = 'test_recipes.txt' , ids_file = 'test_ids.csv')
recipes_json, recipe_ids = convert_to_json(recipes_text, ids_text)

In [58]:
len(recipe_json)

2